# Municipalities
This notebook has the purpose to map all municipalities to their coordinates, as the weather will be queried by the coordinates.

In [11]:
import json
import requests as rq
import pandas as pd

In [101]:
credentials = {}
try:
    with open("../Ressources/credentials.json") as file:
        credentials = json.load(file)
except FileNotFoundError:
    print("Credentials file not found")

In [102]:
# Sample Request with API Ninjas geocoding
municipality = 'Kappel am Albis'
api_url = f'https://api.api-ninjas.com/v1/geocoding?city={municipality}&country=Switzerland'
response = rq.get(api_url + municipality, headers={'X-Api-Key': credentials['ApiNinjasKey']})
if response.status_code == rq.codes.ok:
    print(response)
else:
    print("Error:", response.status_code, response.text)


<Response [200]>


In [103]:
response.json()[0]

{'name': 'Kappel am Albis',
 'latitude': 47.2288482,
 'longitude': 8.5251335,
 'country': 'CH',
 'state': 'Zurich'}

In [109]:
## Load municipalities
municipalities = pd.read_excel('../data_engineering/Data/municipalities/be-b-00.04-agv-20050313.xlsx', sheet_name='Gemeindeliste-Liste d. communes')

In [67]:
new_mun_coordinates = municipalities['GDENAME'].tolist()
problems = []
len_todo = len(new_mun_coordinates)
for i, municipality in enumerate(new_mun_coordinates):
    api_url = f'https://api.api-ninjas.com/v1/geocoding?city={municipality}&country=Switzerland'
    response = rq.get(api_url + municipality, headers={'X-Api-Key': credentials['ApiNinjasKey']})
    if response.status_code == rq.codes.ok:
        if len(response.json())== 0:
            problems.append(municipality)
            print("Error with ", municipality)
            continue
        name = response.json()[0].get("name")
        latitude = response.json()[0].get("latitude")
        longitude = response.json()[0].get("longitude")
        new_mun_coordinates.append([name, latitude, longitude])
    else:
        print("Error:", response.status_code, response.text)
        problems.append(municipality)
    if i %50== 0:
        print(i, " of ", len_todo, " done")

columns = ['Municipality', 'Latitude', 'Longitude']
df_mun_coordinates = pd.DataFrame.from_records(new_mun_coordinates, columns=columns)
df_mun_coordinates.to_csv('../data_engineering/Data/municipality_to_latitude_longitude.csv', index=False)
pd.DataFrame(problems).to_csv('../data_engineering/Data/municipalities_coordinates_problems.csv', index=False)

0  of  1228  done
Error with  Oberhofen (AG)
100  of  1228  done
Error with  Willisau Land
Error with  Rüti (GL)
Error with  Riom-Parsonz
150  of  1228  done
Error with  Reckingen-Gluringen
200  of  1228  done
Error with  Villaz-Saint-Pierre
Error with  Gettnau
250  of  1228  done
Error with  Niedererlinsbach
300  of  1228  done
Error with  Niederried bei Kallnach
Error with  Obererlinsbach
Error with  Isorno
350  of  1228  done
400  of  1228  done
Error with  Casti-Wergenstein
450  of  1228  done
Error with  Tinizong-Rona
Error with  Unterbözberg
500  of  1228  done
Error with  Willisau Stadt
Error with  Wiesen (GR)
Error with  Schwanden (GL)
Error with  Münster-Geschinen
550  of  1228  done
Error with  St. Antönien Ascharina
600  of  1228  done
Error with  Heinrichswil-Winistorf
650  of  1228  done
700  of  1228  done
Error with  Tüscherz-Alfermée
750  of  1228  done
800  of  1228  done
850  of  1228  done
Error with  Klosters-Serneus
Error with  Thielle-Wavre
900  of  1228  done
Err

## Validation
- added the "problematic" municipalities manually
- NO municipality twice in the list, otherwise delete
- join with [municipalities](data_engineering/Data/municipalities/be-b-00.04-agv-20050313.xlsx) as this is the main file given by the swiss government.

In [120]:
df_final_mun = pd.read_csv('../data_engineering/Data/municipalities/municipality_to_latitude_longitude.csv')
df_main_mun = pd.read_excel('../data_engineering/Data/municipalities/be-b-00.04-agv-20050313.xlsx', sheet_name='Gemeindeliste-Liste d. communes')

In [125]:
df_final_mun[df_final_mun.duplicated(['Municipality'])]

,Municipality,Latitude,Longitude


In [130]:
df_main_mun = df_main_mun['GDENAME']

KeyError: 'GDENAME'

In [132]:
df_main_mun = df_main_mun.to_frame()

In [133]:
df_merged = df_main_mun.merge(df_final_mun, left_on='GDENAME', right_on='Municipality', how='left')

In [142]:
df_merged = df_merged[['Municipality', 'Latitude', 'Longitude']]
df_merged.to_csv('../data_engineering/Data/municipalities/municipality_coordinates.csv')

In [138]:
nan_rows = df_merged[df_merged['Latitude'].isna() | df_merged['Longitude'].isna()]
nan_rows

,GDENAME,Municipality,Latitude,Longitude
1363,Suraua,NaN,NaN,NaN


Suraua was updated directly in the data_engineering/Data/municipalities/municipality_coordinates.csv file!